In [252]:
%load_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from torchvision import datasets
from sklearn.metrics import confusion_matrix
from tqdm import tqdm

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [248]:
train_dataset = datasets.MNIST(root='./data', train=True, download=True)
test_dataset = datasets.MNIST(root='./data', train=False, download=True)

train_imgs = train_dataset.data.numpy()
train_labels = train_dataset.targets.numpy()
test_imgs = test_dataset.data.numpy()
test_labels = test_dataset.targets.numpy()

batch = 1000
train_imgs = train_imgs[:batch]
train_labels = train_labels[:batch]

In [249]:
from utils.BP_neural_utils import bp_neural as bp
from utils.BP_neural_utils import *

In [250]:
a = bp_neural(train_imgs,train_labels,n_hidden=100)

Train_images shape: (1000, 28, 28)
Train_lables shape: (1000,)
Input layer nodes: None
Hidden layer nodes: 100
Output layer nodes: None


In [251]:
a.normalize_images(inplace=True)
a.one_hot(inplace=True)
a.init_params()
a.front_prop()

Train_images shape: (784, 1000)
Train_lables shape: (1000,)
Input layer nodes: 784
Hidden layer nodes: 100
Output layer nodes: None
Train_images shape: (784, 1000)
Train_lables shape: (10, 1000)
Input layer nodes: 784
Hidden layer nodes: 100
Output layer nodes: 10


In [268]:
imgs = a.train_imgs
labels = a.train_labels
w1,b1,w2,b2 = a.w1,a.b1,a.w2,a.b2
z1,a1,z2,a2 = a.z1,a.a1,a.z2,a.a2

batch = labels.shape[1]
dz2 = -(labels-a2)
dw2 = 1/batch * np.dot(dz2,a1.T).T
db2 = 1/batch * np.sum(dz2,axis=1,keepdims=True)
da1 = np.dot(dz2.T,w2.T).T
dz1 = da1*a.sigmoid_d(z1)
dw1 = 1/batch * np.dot(dz1,imgs.T).T
db1 = 1/batch * np.sum(dz1,axis=1,keepdims=True)

lr = 1
w1 = w1 - lr * dw1
b1 = b1 - lr * db1
w2 = w2 - lr * dw2
b2 = b2 - lr * db2

print(dw2.max())
a.w1,a.b1,a.w2,a.b2 = w1,b1,w2,b2

a.front_prop()

np.sum(a.predict(imgs) == train_labels)/60000

0.021538456050729946


0.0104

In [213]:
a2

array([[1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [0.99999989, 0.99999476, 0.99999994, ..., 0.99999973, 0.99999992,
        0.99999891],
       ...,
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ],
       [1.        , 1.        , 1.        , ..., 1.        , 1.        ,
        1.        ]])

In [170]:
a.w2

array([[ 1.48705970e+00,  2.90009666e-01,  1.28890714e+00,
         1.14578605e+00,  3.93667736e-01,  9.51328496e-01,
         4.61361314e-01,  5.39928047e-01,  1.57975754e-01,
         1.21526632e+00],
       [ 5.08283579e-01,  2.24459449e+00, -2.81938244e-01,
         6.07121206e-01,  2.23279288e-01, -1.09102793e+00,
        -8.87944795e-01,  8.11997085e-01,  6.70125439e-01,
         1.09984994e+00],
       [ 1.06513215e+00, -1.25942793e-01,  8.40549209e-01,
        -5.26602219e-02,  5.10220657e-01,  1.70857282e+00,
         1.50412399e+00,  5.24912782e-01,  8.76825853e-01,
         1.37986008e+00],
       [ 2.19780440e+00,  1.16545777e+00,  1.06633364e+00,
         1.56041113e+00,  1.03280227e+00,  1.56024939e+00,
         2.29270712e-01,  1.32113237e+00,  6.22077741e-01,
         1.13891575e+00],
       [-1.70985218e+00,  7.76655922e-01,  1.60527549e+00,
         1.65802541e+00, -1.17896009e+00, -5.69095091e-01,
         1.30791522e-01,  1.20259130e-01, -3.47165067e-01,
         5.